In [1]:
import psycopg2, sqlite3, sys, pytz, pdb
from psycopg2.extensions import AsIs
from datetime import datetime
from uuid import uuid4
import pandas as pd
import numpy as np

In [2]:
def is_dst(aware_dt):
    assert aware_dt.tzinfo is not None
    assert aware_dt.tzinfo.utcoffset(aware_dt) is not None
    return bool(aware_dt.dst())

In [3]:
def createTeam1ResultsMethods():
    try:
        conn = psycopg2.connect("dbname='ncpfast_dev' user='django_dev' host='ncpfast-db.edc.renci.org' port='5432' password='MKeeJn9zEqs2T9wm4V3UX'")
        cur = conn.cursor()
        
        cur.execute("""SET CLIENT_ENCODING TO UTF8""")
        cur.execute("""SET STANDARD_CONFORMING_STRINGS TO ON""")
        cur.execute("""BEGIN""")
        # Create cluster geometry table
        cur.execute("""CREATE TABLE team1_results_methods (
                resultid SERIAL PRIMARY KEY,
                resultuuid uuid NOT NULL,
                samplingfeatureexternalidentifier VARCHAR (255) NOT NULL,
                samplingfeaturecode VARCHAR (50) NOT NULL,
                samplingfeaturename VARCHAR (255) NOT NULL,
                variablenamecv VARCHAR (255) NOT NULL,
                variablecode VARCHAR (255) NOT NULL,
                cv_variable_term VARCHAR (255) NOT NULL,
                datavalue DOUBLE PRECISION NOT NULL,
                resultdatetime TIMESTAMP WITH TIME ZONE,
                datasetcode VARCHAR (50) NOT NULL,
                sitetypecv VARCHAR (255) NOT NULL,
                propertyvalue VARCHAR (255) NOT NULL,
                methodid INTEGER NOT NULL,
                methodcode VARCHAR (50) NOT NULL,
                resulttypecv VARCHAR (25) NOT NULL,
                sampledmediumcv VARCHAR (25),
                statuscv VARCHAR (15),
                unitid INTEGER NOT NULL,
                unitabbreviation VARCHAR (25) NOT NULL
            )""")

        cur.execute("""COMMIT""")
        cur.execute("""ANALYZE team1_results_methods""")

    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if cur is not None:
            cur.close()
        if conn is not None:
            conn.close()

In [4]:
#createTeam1ResultsMethods()

In [5]:
def getTeam1ConcResultsMethods():
    conn = None
    try:
        conn = sqlite3.connect('../RoundOneDB/RoundOneDB_final.db')
        cur = conn.cursor()
        cur.execute("""SELECT q.epa_srcid, q.station_id, q.station_name, q.analyte_name,
                       q.abbreviation, q.cas, q.conc_ppt, q.sample_date, q.lab, g.src_type, g.depth
                       FROM quant_summary q 
                       INNER JOIN geolocation g ON g.EPA_srcID = q.EPA_srcID
                       WHERE q.conc_ppt NOT NULL
                       ORDER BY q.epa_srcid""")

        rows = cur.fetchall()
        cur.close()
        conn.close()
     
        df = pd.DataFrame(columns = ['samplingfeatureexternalidentifier', 'samplingfeaturecode',
                                     'samplingfeaturename', 'variablenamecv', 'variablecode',
                                     'cv_variable_term', 'datavalue', 'resultdatetime', 'datasetcode',
                                     'sitetypecv', 'propertyvalue', 'methodid', 'methodcode', 'resulttypecv',
                                     'sampledmediumcv', 'statuscv', 'unitid','unitabbreviation'])
                                     #'class', 'well_i_rat', 'intk_loc']) 
        for row in rows:
            year,month,day = row[7].split('-')
            naive = datetime(int(year),int(month),int(day))
            eastern = pytz.timezone('America/New_York')
            aware = eastern.localize(naive, is_dst=None)
            resultdatetime = aware.replace(hour=12, minute=0, second=0)
            #is_dst(resultdatetime)
            
            datasetcode = 'T1Data_'+str(row[8])
            src_type = row[9]
            
            if src_type == 'G':
                sitetypecv = 'Well'
                methodid = 10003
                methodcode = 'grab-gw'
            elif src_type == 'S':
                sitetypecv = 'Stream'
                methodid = 10005
                methodcode = 'grab-sw'
            else:
                sitetypecv = 'None'
                methodid = NaN
                methodcode = 'None'
                
            resulttypecv = 'Measurement'
            sampledmediumcv = 'Liquid aqueous'
            statuscv = 'Public'
            unitid = 2605
            unitabbreviation = 'ppt'
            
            df = df.append({'samplingfeatureexternalidentifier': row[0],'samplingfeaturecode': row[1],
                            'samplingfeaturename': row[2],'variablenamecv': row[3],'variablecode': row[4],
                            'cv_variable_term': row[5], 'datavalue': row[6],'resultdatetime': resultdatetime,
                            'datasetcode': datasetcode, 'sitetypecv': sitetypecv, 'propertyvalue': row[10],
                            'methodid': methodid, 'methodcode': methodcode, 'resulttypecv': resulttypecv, 
                            'sampledmediumcv': sampledmediumcv, 'statuscv': statuscv, 'unitid': unitid,
                            'unitabbreviation': unitabbreviation},
                            #'class': row[11], 'well_i_rat': row[12], 'intk_loc': row[13]}, 
                           ignore_index = True)
        
        return df
    except Error as e:
        print(e)

In [6]:
#df = getTeam1ConcResultsMethods()
#df.head()

,samplingfeatureexternalidentifier,samplingfeaturecode,samplingfeaturename,variablenamecv,variablecode,cv_variable_term,datavalue,resultdatetime,datasetcode,sitetypecv,propertyvalue,methodid,methodcode,resulttypecv,sampledmediumcv,statuscv,unitid,unitabbreviation
0,11736,NC0105010,"WEST JEFFERSON, TOWN OF",Perfluorobutanesulfonic acid,PFBS,375-73-5,3.399,2019-07-02 12:00:00-04:00,T1Data_Knappe,Well,500,10003,grab-gw,Measurement,Liquid aqueous,Public,2605,ppt
1,11736,NC0105010,"WEST JEFFERSON, TOWN OF",Perfluorohexanesulfonic acid,PFHxS,355-46-4,4.438,2019-07-02 12:00:00-04:00,T1Data_Knappe,Well,500,10003,grab-gw,Measurement,Liquid aqueous,Public,2605,ppt
2,12111,NC0111020,"BLACK MOUNTAIN, TOWN OF",Perfluorooctanoic acid,PFOA,335-67-1,2.200,2019-06-24 12:00:00-04:00,T1Data_Ferguson,Well,505,10003,grab-gw,Measurement,Liquid aqueous,Public,2605,ppt
3,12111,NC0111020,"BLACK MOUNTAIN, TOWN OF",Perfluoroundecanoic acid,PFUnDA,2058-94-8,1.300,2019-06-24 12:00:00-04:00,T1Data_Ferguson,Well,505,10003,grab-gw,Measurement,Liquid aqueous,Public,2605,ppt
4,17170,NC0156025,"OLD FORT, TOWN OF",Perfluorooctanoic acid,PFOA,335-67-1,1.800,2019-06-24 12:00:00-04:00,T1Data_Ferguson,Well,305,10003,grab-gw,Measurement,Liquid aqueous,Public,2605,ppt


In [7]:
#outfile = '../ncpfast-db/team1ConcResultsMethods.csv'
#df.to_csv(outfile, header=True, index=False)

In [8]:
#len(df['datavalue'])

815

In [9]:
def getTeam1RlResultsMethods():
    conn = None
    try:
        conn = sqlite3.connect('../RoundOneDB/RoundOneDB_final.db')
        cur = conn.cursor()
        cur.execute("""SELECT q.epa_srcid, q.station_id, q.station_name, q.analyte_name,
                       q.abbreviation, q.cas, q.rl_ppt, q.sample_date, q.lab, g.src_type, g.depth
                       FROM quant_summary q 
                       INNER JOIN geolocation g ON g.EPA_srcID = q.EPA_srcID
                       WHERE q.rl_ppt NOT NULL
                       ORDER BY q.epa_srcid""")

        rows = cur.fetchall()
        cur.close()
        conn.close()
     
        df = pd.DataFrame(columns = ['samplingfeatureexternalidentifier', 'samplingfeaturecode',
                                     'samplingfeaturename', 'variablenamecv', 'variablecode',
                                     'cv_variable_term', 'datavalue', 'resultdatetime', 'datasetcode',
                                     'sitetypecv', 'propertyvalue', 'methodid', 'methodcode', 'resulttypecv',
                                     'sampledmediumcv', 'statuscv', 'unitid','unitabbreviation'])
                                     #'class', 'well_i_rat', 'intk_loc']) 
        for row in rows:
            year,month,day = row[7].split('-')
            naive = datetime(int(year),int(month),int(day))
            eastern = pytz.timezone('America/New_York')
            aware = eastern.localize(naive, is_dst=None)
            resultdatetime = aware.replace(hour=12, minute=0, second=0)
            #is_dst(resultdatetime)
            
            datasetcode = 'T1Data_'+str(row[8])
            src_type = row[9]
            if src_type == 'G':
                sitetypecv = 'Well'
                methodid = 10003
                methodcode = 'grab-gw'
            elif src_type == 'S':
                sitetypecv = 'Stream'
                methodid = 10005
                methodcode = 'grab-sw'
            else:
                sitetypecv = 'None'
                
            resulttypecv = 'Measurement'
            sampledmediumcv = 'Liquid aqueous'
            statuscv = 'Public'
            unitid = 2793
            unitabbreviation = 'rl_ppt'
            
            df = df.append({'samplingfeatureexternalidentifier': row[0],'samplingfeaturecode': row[1],
                            'samplingfeaturename': row[2],'variablenamecv': row[3],'variablecode': row[4],
                            'cv_variable_term': row[5], 'datavalue': row[6],'resultdatetime': resultdatetime,
                            'datasetcode': datasetcode, 'sitetypecv': sitetypecv, 'propertyvalue': row[10],
                            'methodid': methodid, 'methodcode': methodcode, 'resulttypecv': resulttypecv, 
                            'sampledmediumcv': sampledmediumcv, 'statuscv': statuscv, 'unitid': unitid,
                            'unitabbreviation': unitabbreviation},
                            #'class': row[11], 'well_i_rat': row[12], 'intk_loc': row[13]}, 
                           ignore_index = True)
        
        return df
    except Error as e:
        print(e)

In [16]:
#df = getTeam1RlResultsMethods()
#df.head()

,samplingfeatureexternalidentifier,samplingfeaturecode,samplingfeaturename,variablenamecv,variablecode,cv_variable_term,datavalue,resultdatetime,datasetcode,sitetypecv,propertyvalue,methodid,methodcode,resulttypecv,sampledmediumcv,statuscv,unitid,unitabbreviation
0,11606,NC0103010,"SPARTA, TOWN OF",Perfluoro-2-ethoxypropanoic acid,PEPA,267239-61-2,1.0,2019-07-02 12:00:00-04:00,T1Data_Knappe,Well,329,10003,grab-gw,Measurement,Liquid aqueous,Public,2793,rl_ppt
1,11606,NC0103010,"SPARTA, TOWN OF","1,1,2,2-tetrafluoro-2-(1,2,2,2-tetrafluoro-eth...",NVHOS,N/A,5.0,2019-07-02 12:00:00-04:00,T1Data_Knappe,Well,329,10003,grab-gw,Measurement,Liquid aqueous,Public,2793,rl_ppt
2,11606,NC0103010,"SPARTA, TOWN OF",N-(3-dimethylaminopropan-1-yl)perfluoro-1-hexa...,N-AP-FHxSA,50598-28-2,10.0,2019-07-02 12:00:00-04:00,T1Data_Knappe,Well,329,10003,grab-gw,Measurement,Liquid aqueous,Public,2793,rl_ppt
3,11606,NC0103010,"SPARTA, TOWN OF",Perfluorononanesulfonic acid,PFNS,68259-12-1,2.0,2019-07-02 12:00:00-04:00,T1Data_Knappe,Well,329,10003,grab-gw,Measurement,Liquid aqueous,Public,2793,rl_ppt
4,11606,NC0103010,"SPARTA, TOWN OF",Perfluorooctanoic acid,PFOA,335-67-1,2.0,2019-07-02 12:00:00-04:00,T1Data_Knappe,Well,329,10003,grab-gw,Measurement,Liquid aqueous,Public,2793,rl_ppt


In [17]:
#outfile = '../ncpfast-db/team1RlResultsMethods.csv'
#df.to_csv(outfile, header=True, index=False)

In [18]:
#len(df['datavalue'])

17356

In [13]:
def loadTeam1ResultsMethods(units):
    
    if units.lower() == 'rl_ppt':
        infile = '../ncpfast-db/team1RlResultsMethods.csv'
    elif units.lower() == 'conc_ppt':
        infile = '../ncpfast-db/team1ConcResultsMethods.csv'
    else:
        sys.exit('Incorret units')

    df = pd.read_csv(infile)

    try:
        conn = psycopg2.connect("dbname='ncpfast_dev' user='django_dev' host='ncpfast-db.edc.renci.org' port='5432' password='MKeeJn9zEqs2T9wm4V3UX'")
        cur = conn.cursor()
        cur.execute("""SET CLIENT_ENCODING TO UTF8""")
        cur.execute("""SET STANDARD_CONFORMING_STRINGS TO ON""")
        cur.execute("""BEGIN""")
        
        for index, row in df.iterrows():
            resultuuid = str(uuid4())
            samplingfeatureexternalidentifier = row[0]
            samplingfeaturecode = row[1]
            samplingfeaturename = row[2]
            variablenamecv = row[3]
            variablecode = row[4]
            cv_variable_term = row[5]
            datavalue = row[6]
            resultdatetime = row[7]
            datasetcode = row[8]
            sitetypecv = row[9]
            propertyvalue = row[10]
            methodid = row[11]
            methodcode = row[12]
            resulttypecv = row[13]
            sampledmediumcv = row[14]
            statuscv = row[15]
            unitid = row[16]
            unitabbreviation = row[17]
            
            cur.execute("""INSERT INTO team1_results_methods(resultuuid,samplingfeatureexternalidentifier, 
                                                             samplingfeaturecode,samplingfeaturename,
                                                             variablenamecv,variablecode,cv_variable_term,
                                                             datavalue,resultdatetime,datasetcode,sitetypecv,
                                                             propertyvalue,methodid,methodcode,resulttypecv,
                                                             sampledmediumcv,statuscv,unitid,unitabbreviation)
                            VALUES (%(vresultuuid)s,%(vsamplingfeatureexternalidentifier)s,%(vsamplingfeaturecode)s,
                                    %(vsamplingfeaturename)s,%(vvariablenamecv)s,%(vvariablecode)s,
                                    %(vcv_variable_term)s,%(vdatavalue)s,%(vresultdatetime)s,%(vdatasetcode)s,
                                    %(vsitetypecv)s,%(vpropertyvalue)s,%(vmethodid)s,%(vmethodcode)s,
                                    %(vresulttypecv)s,%(vsampledmediumcv)s,%(vstatuscv)s,%(vunitid)s,
                                    %(vunitabbreviation)s)""",
                       {'vresultuuid':resultuuid,
                        'vsamplingfeatureexternalidentifier':samplingfeatureexternalidentifier,
                        'vsamplingfeaturecode':samplingfeaturecode,
                        'vsamplingfeaturename':samplingfeaturename,'vvariablenamecv':variablenamecv,
                        'vvariablecode':variablecode,'vcv_variable_term':cv_variable_term,'vdatavalue':datavalue,
                        'vresultdatetime':resultdatetime,'vdatasetcode':datasetcode,'vsitetypecv':sitetypecv,
                        'vpropertyvalue':propertyvalue,'vmethodid':methodid,'vmethodcode':methodcode,
                        'vresulttypecv':resulttypecv,'vsampledmediumcv':sampledmediumcv,'vstatuscv':statuscv,
                        'vunitid': unitid,'vunitabbreviation': unitabbreviation})

        conn.commit()
        cur.execute("""ANALYZE team1_results_methods""")
        
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if cur is not None:
            cur.close()
        if conn is not None:
            conn.close()      


In [19]:
#loadTeam1ResultsMethods('conc_ppt')
#loadTeam1ResultsMethods('rl_ppt')

In [15]:
def insertResultsMethods():
    try:
        conn = psycopg2.connect("dbname='ncpfast_dev' user='django_dev' host='ncpfast-db.edc.renci.org' port='5432' password='MKeeJn9zEqs2T9wm4V3UX'")
        cur = conn.cursor()
        
        cur.execute("""SET CLIENT_ENCODING TO UTF8""")
        cur.execute("""SET STANDARD_CONFORMING_STRINGS TO ON""")
        cur.execute("""BEGIN""")
        cur.execute("""SELECT samplingfeaturecode FROM team1_sampling_features""")
        frows = cur.fetchall()
        
        for frow in frows:
            samplingfeaturecode = frow[0]
            cur.execute("""SELECT DISTINCT resultdatetime FROM team1_results_methods 
                           WHERE samplingfeaturecode = %(vsamplingfeaturecode)s""",
                        {'vsamplingfeaturecode': samplingfeaturecode})
            resultdatetimes = cur.fetchone()[0]
            
            begin_datetime = resultdatetime.replace(hour=0, minute=0, second=0)
            end_datetime = resultdatetime.replace(hour=23, minute=59, second=59)
            validdatetime = resultdatetime.replace(hour=0, minute=0, second=0)
            endvaliddatetime = resultdatetime.replace(hour=23, minute=59, second=59)
            valuedatetime = resultdatetime

            cur.execute("""SELECT DISTINCT methodid FROM team1_results_methods 
                           WHERE samplingfeaturecode = %(vsamplingfeaturecode)s""",
                        {'vsamplingfeaturecode': samplingfeaturecode})
            methodid = cur.fetchone()[0]
            
            cur.execute("""INSERT INTO actions(begin_datetime,end_datetime,methodid)
                            VALUES (%(vbegin_datetime)s,%(vend_datetime)s,%(vmethodid)s)
                            RETURNING actionid""",
                        {'vbegin_datetime': begin_datetime,'vend_datetime': end_datetime,'vmethodid': methodid})
            actionid = cur.fetchone()[0]

            cur.execute("""SELECT samplingfeatureid FROM sampling_features 
                            WHERE samplingfeaturecode = %(vsamplingfeaturecode)s""",
                        {'vsamplingfeaturecode':samplingfeaturecode})
            samplingfeatureid = cur.fetchone()[0]
            
            cur.execute("""INSERT INTO feature_actions(actionid,samplingfeatureid)
                            VALUES (%(vactionid)s,%(vsamplingfeatureid)s)
                            RETURNING featureactionid""",
                        {'vactionid': actionid,'vsamplingfeatureid': samplingfeatureid})
            featureactionid = cur.fetchone()[0]
            
            cur.execute("""SELECT * FROM team1_results_methods 
                           WHERE samplingfeaturecode = %(vsamplingfeaturecode)""",
                        {'vsamplingfeaturecode':samplingfeaturecode})
            rows = cur.fetchall()
            
            for row in rows:
                resultidin = row[0]
                resultuuid = row[1]
                samplingfeatureexternalidentifier = row[2]
                samplingfeaturecode = row[3]
                samplingfeaturename = row[4]
                variablenamecv = row[5]
                variablecode = row[6]
                cv_variable_term = row[7]
                datavalue = row[8]
                #resultdatetime = row[9].strftime("%Y-%m-%d %H:%M:%S")
                resultdatetime = row[9]
                datasetcode = row[10]
                sitetypecv = row[11]
                propertyvalue = row[12]
                methodid = row[13]
                methodcode = row[14]
                resulttypecv = row[15]
                sampledmediumcv = row[16]
                statuscv = row[17]
                unitid = row[18]
                unitabbreviation = row[19]
                
                cur.execute("""SELECT datasetid FROM datasets 
                               WHERE datasetcode = %(vdatasetcode)s""",
                            {'vdatasetcode':datasetcode})
                datasetid = cur.fetchone()[0]
                
                cur.execute("""SELECT variableid FROM variables 
                               WHERE variablecode = %(vvariablecode)s""",
                            {'vvariablecode':variablecode})
                variableidid = cur.fetchall()                
            
                cur.execute("""INSERT INTO results(resultuuid,resultdatetime,featureactionid,resulttypecv,
                                                   sampledmediumcv,statuscv,unitid,variableid,validdatetime,
                                                   endvaliddatetime)
                                VALUES (%(vresultuuid)s,%(vresultdatetime)s,%(vfeatureactionid)s,
                                        %(vresulttypecv)s,%(vsampledmediumcv)s,%(vstatuscv)s,%(vunitid)s,
                                        %(vvariableid)s,%(vvaliddatetime)s,%(vendvaliddatetime)s)
                                RETURNING resultid""",
                           {'vresultuuid':resultuuid,'vresultdatetime':resultdatetime,
                            'vfeatureactionid':featureactionid,'vresulttypecv':resulttypecv,
                            'vsampledmediumcv':sampledmediumcv,'vstatuscv':statuscv,'vunitid': unitid,
                            'vvariableid':variableid,'vvaliddatetime':validdatetime,
                            'vendvaliddatetime':endvaliddatetime})
                resultid = cur.fetchone()[0]
            
                cur.execute("""INSERT INTO measurement_result_values(datavalue,valuedatetime,resultid)
                                VALUES (%(vdatavalue)s,%(vvaluedatetime)s,%(vresultid)s)""",
                           {'vdatavalue':datavalue,'vvaluedatetime':valuedatetime,'vresultid':resultid})

                cur.execute("""INSERT INTO dataset_results(resultid,datasetid)
                                VALUES (%(vresultid)s,%(vdatasetid)s)""",
                            {'vresultid':resultid,'vdatasetid':datasetid})
                
        #conn.commit()
        #cur.execute("""ANALYZE sampling_features""")
        
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if cur is not None:
            cur.close()
        if conn is not None:
            conn.close()  